In [1]:
import numpy as np
import random
import copy

In [2]:
class Individual:
    
    def __init__(self, mentoriZauzetost, zeljeStudenata):
        self.mentoriZauzetost = copy.deepcopy(mentoriZauzetost)
        self.zeljeStudenata = copy.deepcopy(zeljeStudenata)
        self.brMentora = len(mentoriZauzetost)
        self.brStudenata = len(zeljeStudenata)
        self.maxBrojStudenata = sum(mentoriZauzetost)
        
        if self.maxBrojStudenata < self.brStudenata:
            print('Kapaciteti popunjeni! Nije moguce dodeliti mentora svakom studentu!')
        
        studenti = [i for i in range(0, self.maxBrojStudenata)]
        random.shuffle(studenti)
        mentori = []
        br=0
        for i in range(self.brMentora):
            for j in range(self.mentoriZauzetost[i]):
                if br + j > self.brStudenata:
                    break
                mentori.append(i)
            br += j
        self.code = []
        for i in range(self.brStudenata):
            self.code.append((studenti[i], mentori[i]))
        self.fitness = self.fitnessFunction()
        
    def fitnessFunction(self):
        val = 0
        for i in range(self.brStudenata):
            val += self.geneFitness(i)
        return val
    
    def geneFitness(self, i):
        (student, mentor) = self.code[i]
        return self.zeljeStudenata[student][mentor]
    
    def __lt__(self, other):
        return self.fitness < other.fitness

In [3]:
def mutation(individual, mutation_rate):
    
    if random.uniform(0,1) < mutation_rate:
        i = random.randrange(individual.brStudenata)
        j = random.randrange(individual.brStudenata)
        tmp = individual.code[i][0]
        individual.code[i] = (individual.code[j][0], individual.code[i][1])
        individual.code[j] = (tmp, individual.code[j][1])
    

In [4]:
def selection(population, tournament_size):
    val = float('inf')
    bestInd = -1
    for _ in range(tournament_size):
        i = random.randrange(len(population))
        if population[i].fitness < val:
            val = population[i].fitness
            bestInd = i
    return bestInd

In [5]:
def crossover(parent1, parent2, child1):
    
    n = parent1.brStudenata
    dodeljeni = set()
    nisuDodeljenji = set()
    for i in range(n):
        nisuDodeljenji.add(parent1.code[i][0])
    for i in range(n):
        child1.code[i] = (-1, child1.code[i][1])
    for i in range(n):
        if parent1.geneFitness(i) < parent2.geneFitness(i) and parent1.code[i][0] not in dodeljeni:
            child1.code[i] = (parent1.code[i][0], child1.code[i][1])
            dodeljeni.add(child1.code[i][0])
            nisuDodeljenji.remove(child1.code[i][0])
        elif parent1.geneFitness(i) > parent2.geneFitness(i) and parent2.code[i][0] not in dodeljeni:
            child1.code[i] = (parent2.code[i][0], child1.code[i][1])
            dodeljeni.add(child1.code[i][0])
            nisuDodeljenji.remove(child1.code[i][0])
    for i in range(n):
        if child1.code[i][0] == -1:
           
            child1.code[i] = (random.sample(nisuDodeljenji, 1)[0], child1.code[i][1])
            
            nisuDodeljenji.remove(child1.code[i][0])
    child1.fitness = child1.fitnessFunction()
    if child1.fitness > parent1.fitness:
        child1 = parent1
    if child1.fitness > parent2.fitness:
        child1 = parent2
        

In [6]:
mentoriZauzetost = [2, 3, 2, 1, 3]
zeljeStudenata = [
    [0, 1, 2,3, 4],
    [2, 3,2, 0, 1],
    [0, 2,4, 1, 3],
    [2, 4,3,0, 1],
    [4, 3,1, 0, 2],
    [1,4, 2,3, 0],
    [0, 1, 2,3, 4],
    [2, 3,2, 0, 1],
    [0, 2,4, 1, 3],
    [2, 4,3,0, 1],
    [0,2,3, 4,1]
    
]


In [7]:
population_size = 512
mutation_rate = 0.1
tournament_size = 6
max_iter = 9
elitism_size = 20
population = [Individual(mentoriZauzetost, zeljeStudenata) for _ in range(population_size)]
# for i in range(population_size):
#     print(population[i].code)
new_population = []
for iteration in range(max_iter):
    population.sort()
    new_population = []
    if population[0].fitness == 0:
        print("Svakom studentu je ispunjena prva zelja!")
        break
    for i in range(elitism_size):
        new_population.append(population[i])
    for i in range(elitism_size, population_size, 2):
        p1 = selection(population, tournament_size)
        p2 = selection(population, tournament_size)
        
        child = Individual(mentoriZauzetost, zeljeStudenata)
        crossover(population[p1], population[p2], child)
        
        mutation(child, mutation_rate)
        
        child.fitness = child.fitnessFunction()
        new_population.append(child)
    population = new_population

population.sort()

print("najbolje resenje je:")
print(population[0].code)
print(population[0].fitness)
    

najbolje resenje je:
[(10, 0), (8, 0), (2, 1), (6, 1), (0, 1), (7, 2), (4, 2), (1, 3), (5, 4), (3, 4), (9, 4)]
9
